In [1]:
# Importing the 'os' module to interact with the operating system (e.g., environment variables, file paths)
import os  

# Importing the 'load_dotenv' function from 'dotenv' package to load environment variables from a .env file
from dotenv import load_dotenv  

# Importing the 'OpenAI' class from the 'openai' library to interact with OpenAI API services
from openai import OpenAI  


In [4]:
# Load all environment variables from the .env file into the system environment
load_dotenv()

# Retrieve the OpenAI API key stored in the environment variable 'OPENAI_API_KEY'
api_key = os.getenv('OPENAI_API_KEY')


In [5]:
# Create an instance of the OpenAI client to interact with the API
openai = OpenAI()


In [19]:
# Define a multi-line string (triple quotes) that serves as the system/user prompt for the LLM
prompt = '''You are a Cost-Effective Travel Assistant.  

Your role is to plan trips for users based on their inputs: trip duration, budget, and group size.  

You must optimize **accommodation, transportation, and sightseeing recommendations** so that the entire plan stays within budget.  

Guidelines:
- Always stay funny, witty, and conversational (like a travel buddy with a sense of humor).  
- Be practical but also entertaining—make the user smile while giving useful advice.  
- Suggest cost-saving hacks (e.g., public transport, budget hotels, local food stalls).  
- Provide a structured trip plan (Day-wise if needed).  
- Ensure recommendations are realistic and feasible within the budget.  
- Add quirky remarks, emojis, or jokes to keep the tone engaging.  
- And also make sure to tell them if the budget is not enough, and suggest how much extra money they need to add to make the trip possible, otherwise inform them no trip can be planned.  
'''


In [20]:
# Define a function 'gpt' that takes two inputs:
# msg -> the new user message
# his -> the conversation history (list of messages)
def gpt(msg, his):
    # Construct the full message list:
    # Start with the system prompt, then add conversation history, then the new user message
    message = [{'role': 'system', 'content': prompt}] + his + [{'role': 'user', 'content': msg}]
    
    # Call the OpenAI Chat API with streaming enabled
    result = openai.chat.completions.create(
        model='gpt-4o-mini',  # The model being used
        messages=message,     # The constructed conversation
        stream=True           # Enable streaming response
    )
    
    # Initialize an empty string to collect the streamed response
    p = ''
    
    # Iterate through the streamed chunks of response
    for i in result:
        # Append new content if available (avoid None values with 'or ""')
        p += i.choices[0].delta.content or ''
        
        # Yield the progressively built response (generator style)
        yield p


In [9]:
# Import the Gradio library for creating a simple web-based user interface
import gradio as gr


In [24]:
# Create a Gradio ChatInterface where 'gpt' function handles the conversation
# 'type="messages"' ensures the history is passed in the format expected by the gpt() function
gui = gr.ChatInterface(fn=gpt, type='messages')

# Launch the Gradio app with sharing enabled (generates both local and public URLs)
app, local_url, share_url = gui.launch(share=True)

# Print the local URL (accessible only on the same machine)
print("Local URL:", local_url)

# Print the shareable public URL (accessible from anywhere)
print("Share URL:", share_url)


* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://1e195526e6b1c13829.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Local URL: http://127.0.0.1:7867/
Share URL: https://1e195526e6b1c13829.gradio.live
